# Convert DICOM images to niftys in BIDS structure

### Author: Olaf Borghi

Parts of this notebook are coded in Python, other parts in Bash. Bash code is indicated by %%bash at the beginning of a cell. 
**Tutorial**: https://unfmontreal.github.io/Dcm2Bids/

**System:** Ubuntu 22.04.1

**Environment:** neuroenv (custom conda environment)

**Dependencies of dcm2bids (installed in the conda environment)**: 
- dcm2niix (can be installed in a conda environment by conda install -c conda-forge dcm2niix)
- dcm2bids (can be installed in a conda environment by conda install -c conda-forge dcm2bids)
- python>=3.7

**Other libraries installed**
- jupyter lab (conda install -c conda-forge jupyterlab / conda install -c conda-forge nb_conda_kernels)
- numpy, pip, pathlib, json, 
- Tree (sudo apt-get tree)

## Setup

In [1]:
# import modules, make sure dependencies are installed in the environment
import os
import json
import dcm2bids
import numpy as np
from pathlib import Path

In [2]:
os.getcwd()

'/mnt/o/AON2bids'

In [3]:
# specify working directory in Python
work_dir=Path('/mnt/o/AON2bids')
# change to working directory to work_dir
os.chdir(work_dir)

In [5]:
%%bash
# create scaffold and store it in folder bids_scaffold
dcm2bids_scaffold -o 03_bids_scaffold

In [4]:
# navigate to the scaffold folder
os.chdir(work_dir / "03_bids_scaffold")
os.getcwd()

'/mnt/o/AON2bids/03_bids_scaffold'

### Import/locate/store the data

You should either have the data available on your local machine already in a folder, or download the MRI data you would like to convert with !wget or similar tools. Make sure the data is also already unzipped, or unzip it before proceeding with the next steps.

## Create the config file

First, use the dcm2bids helper function to create a temporary folder were the dicoms are converted to niftys, so that you can read out information to create the config file.

In [9]:
%%bash
# lets take a look at the helper function
dcm2bids_helper --help

usage: dcm2bids_helper [-h] -d DICOM_DIR [DICOM_DIR ...] [-o OUTPUT_DIR]
                       [--force]

helper module

options:
  -h, --help            show this help message and exit
  -d DICOM_DIR [DICOM_DIR ...], --dicom_dir DICOM_DIR [DICOM_DIR ...]
                        DICOM files directory.
  -o OUTPUT_DIR, --output_dir OUTPUT_DIR
                        Output BIDS directory. (Default: /mnt/o/AON2bids/03_bids_scaffold)
  --force               Force command to overwrite existing output files.

Documentation at https://github.com/unfmontreal/Dcm2Bids


In [6]:
%%bash
# specify the data directory
data_dir='/mnt/o/AON2bids/01_data/mri/mri/'

# run the helper function on sample data from one subject [in this case subject with the ID EV01]
dcm2bids_helper -d ${data_dir}EV01

Example in: /mnt/o/AON2bids/03_bids_scaffold/tmp_dcm2bids/helper


**Data and exemplary task**

A temporary folder containing information on your scans was made. In this folder, you can find a)converted nifty images for each acquisition and b).json files containing information for each acqusition.

Note that a file was created for every file in the folder, but for now I am interested in the scans from an ACTION run, in which participants were instructed to move there hand when a hand-symbol was showing on the screen, and to move their mouth, when a mouth symbol was displayed. I am also interested in two runs of an Action Observation task, in which participants watched either 1) a dog or 2) a human performing a a) transitive or b) intransitive condition, as well as scrambled versions of these conditions. 

In addition I am interested in the T1 anatomical image and in the field map (one for all functional scans).

You can open the JSON files from the temporary directory either directly in a new tab in Jupyter Lab or with any other text/code editor. Make sure to localize the correct file.

### Populate a .json config file

In this step, I create a .JSON config file in the bids_scaffold/code folder (you can also choose any other folder). I will do this within this notebook, but you can of course do this with every other code/text editor and don't need to do it in python. For a guide on how to work with .json files in Python, e.g. refer to https://pythonexamples.org/python-write-json-to-file/.

We want to BIDSify the following data.
- T1 anatomical image
- ACTION run 
- AON run 1
- AON run 2
- Fielmap 

So first we have to find the corresponding files for each run/task/protocol in the temporary files that we created with the helper function in the folder \tmp_dcm2bids\helper.

Please refer to these pages to see how to populate the config file:

https://unfmontreal.github.io/Dcm2Bids/docs/how-to/create-config-file/ 

https://unfmontreal.github.io/Dcm2Bids/docs/tutorial/first-steps/

And for more general starting information on the BIDS format, refer to:

https://bids-standard.github.io/bids-starter-kit/tutorials/annotation.html

The config file basically has the structure of a dictonary and the information it contains should uniquely match the acquisition you target, in many cases the ```"SeriesDescription"``` in the temporary .json file will be a good indicator for the target acquisition.

In [7]:
# create the dictonary with the information from the temporary files
dcm2bids_dict = {
  "descriptions": [
    {
      "dataType": "anat",
      "modalityLabel": "T1w",
      "criteria": {
        "SeriesDescription": "t1_mprage_sag_p2_iso",
        "EchoTime": 0.00229
      },
      "sidecarChanges": {
        "ProtocolName": "t1_mprage_sag_p2_iso"
      }
    },
    {
      "dataType": "func",
      "modalityLabel": "bold",
      "customLabels": "task-action",
      "criteria": {
        "SeriesDescription": "ACTION_run_HR_222_tr1200_te34"
      },
      "sidecarChanges": {
        "TaskName": "action"
      }
    },
    {
      "dataType": "func",
      "modalityLabel": "bold",
      "customLabels": "task-AON_run-01",
      "criteria": {
        "SeriesDescription": "AONH_run1_HR_222_tr1200_te34"
      },
      "sidecarChanges": {
        "TaskName": "AON"
      }
    },
    {
      "dataType": "func",
      "modalityLabel": "bold",
      "customLabels": "task-AON_run-02",
      "criteria": {
        "SeriesDescription": "AONH_run2_HR_222_tr1200_te34"
      },
      "sidecarChanges": {
        "TaskName": "AON"
      }
    },
    {
      "dataType": "fmap",
      "modalityLabel": "phasediff",
      "criteria": {
        "SeriesDescription": "gre_field_mapping_bigFOV",
        "EchoTime1": 0.00492,
        "EchoTime2": 0.00738,
        "ImageType": ["ORIGINAL", "PRIMARY", "P", "ND", "PHASE"]
      },
      "IntendedFor": [1,2,3],
      "sidecarChanges": {
        "EchoTime1": 0.00492,
        "EchoTime2": 0.00738
        }
      },
    {
      "dataType": "fmap",
      "modalityLabel": "magnitude1",
      "criteria": {
        "SeriesDescription": "gre_field_mapping_bigFOV",
        "EchoNumber": 1,
        "EchoTime": 0.00492,
        "ImageType": ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"]
      },
      "IntendedFor": [1,2,3],
      "sidecarChanges": {
        "EchoTime": 0.00492
        }
      },
    {
      "dataType": "fmap",
      "modalityLabel": "magnitude2",
      "criteria": {
        "SeriesDescription": "gre_field_mapping_bigFOV",
        "EchoNumber": 2,
        "EchoTime": 0.00738,
        "ImageType": ["ORIGINAL", "PRIMARY", "M", "ND", "NORM"]
      },
      "IntendedFor": [1,2,3],
      "sidecarChanges": {
        "EchoTime": 0.00738
        }
    }
  ]
}

In [19]:
# change directory to the code scaffold folder
os.chdir(work_dir / "03_bids_scaffold/code")

In [20]:
# create the json file from the dict and store it in the wd
jsonString = json.dumps(dcm2bids_dict,
                        sort_keys=True,
                        indent=4,
                        separators=(',', ': '))
jsonFile = open("dcm2bids_config.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [21]:
%%bash
# see if the .json file is displayed correctly
cat dcm2bids_config.json

{
    "descriptions": [
        {
            "criteria": {
                "EchoTime": 0.00229,
                "SeriesDescription": "t1_mprage_sag_p2_iso"
            },
            "dataType": "anat",
            "modalityLabel": "T1w",
            "sidecarChanges": {
                "ProtocolName": "t1_mprage_sag_p2_iso"
            }
        },
        {
            "criteria": {
                "SeriesDescription": "ACTION_run_HR_222_tr1200_te34"
            },
            "customLabels": "task-action",
            "dataType": "func",
            "modalityLabel": "bold",
            "sidecarChanges": {
                "TaskName": "action"
            }
        },
        {
            "criteria": {
                "SeriesDescription": "AONH_run1_HR_222_tr1200_te34"
            },
            "customLabels": "task-AON_run-01",
            "dataType": "func",
            "modalityLabel": "bold",
            "sidecarChanges": {
                "TaskName": "AON"
            }
       

**As soon as the config file is ready, we can delete the temporary files that we created with the helper function**

In [22]:
%%bash
tempdir='/mnt/o/AON2bids/03_bids_scaffold/tmp_dcm2bids'
# delete temporary files
rm -r ${tempdir}

## Ready to go - run dcm2bids

Bash code adapted from https://danieljwilson.com/science/2019/02/18/dicom-bids/

Now we get to the easy part. Store/download/unzip the data in the bids_scaffold/sourcedata folder and you are good to run the following script.
In my case, the folders were named EV01, EV02 .. EV40 for each subject. Therefore, I had to make a if-loop nested in a for loop to add a zero to the subject/foldername for subjects 1-9. This may not be necessary in your case.

In [5]:
%%bash
# let's quickly look at the help function
dcm2bids --help

usage: dcm2bids [-h] -d DICOM_DIR [DICOM_DIR ...] -p PARTICIPANT [-s SESSION]
                -c CONFIG [-o OUTPUT_DIR] [--forceDcm2niix] [--clobber]
                [-l {DEBUG,INFO,WARNING,ERROR,CRITICAL}]

Reorganising NIfTI files from dcm2niix into the Brain Imaging Data Structure

options:
  -h, --help            show this help message and exit
  -d DICOM_DIR [DICOM_DIR ...], --dicom_dir DICOM_DIR [DICOM_DIR ...]
                        DICOM directory(ies).
  -p PARTICIPANT, --participant PARTICIPANT
                        Participant ID.
  -s SESSION, --session SESSION
                        Session ID.
  -c CONFIG, --config CONFIG
                        JSON configuration file (see example/config.json).
  -o OUTPUT_DIR, --output_dir OUTPUT_DIR
                        Output BIDS directory. (Default: /mnt/o/AON2bids/03_bids_scaffold)
  --forceDcm2niix       Overwrite previous temporary dcm2niix output if it exists.
  --clobber             Overwrite output if it exists.
  -l {D

In [6]:
%%bash

bidsdir='/mnt/o/AON2bids/03_bids_scaffold'
mkdir ${bidsdir}/output

# # convert dicoms to BIDS format dataset
# # MULTIPLE SUBJECTS
# # more general code if your sourcefolders are named 1,2,3 ... according to the subject id
# for subject in {1,2}; do
# 	dcm2bids \
# 	    -d $bidsdir/sourcedata/${subject} \
# 	    -p ${subject} \
# 	    -c $bidsdir/code/dcm2bids_config.json \
# 	    -o $bidsdir
# done

# convert dicoms to BIDS format dataset
# MULTIPLE SUBJECTS
# code if your folders are named differently, as in my case (EV01, EV02 .. EV40)
for subject in {3,33}; do # quickly changed to try just for one sbj (for subject in {1:40}; do)
    if [[ $subject -lt 9 ]]; then
        dcm2bids \
        -d /mnt/o/AON2bids/01_data/mri/mri/EV0${subject} \
        -p 0${subject} \
        -c $bidsdir/code/dcm2bids_config.json \
        -o $bidsdir/output
    elif [[ $subject -gt 9 ]]; then
        dcm2bids \
        -d /mnt/o/AON2bids/01_data/mri/mri/EV${subject} \
        -p ${subject} \
        -c $bidsdir/code/dcm2bids_config.json \
        -o $bidsdir/output
    fi
done

mkdir: cannot create directory ‘/mnt/o/AON2bids/03_bids_scaffold/output’: File exists
INFO:dcm2bids.dcm2bids:--- dcm2bids start ---
INFO:dcm2bids.dcm2bids:OS:version: Linux-5.10.16.3-microsoft-standard-WSL2-x86_64-with-glibc2.35
INFO:dcm2bids.dcm2bids:python:version: 3.11.0 | packaged by conda-forge | (main, Oct 25 2022, 06:24:40) [GCC 10.4.0]
INFO:dcm2bids.dcm2bids:dcm2bids:version: 2.1.7
INFO:dcm2bids.dcm2bids:dcm2niix:version: v1.0.20220720
INFO:dcm2bids.dcm2bids:participant: sub-03
INFO:dcm2bids.dcm2bids:session: 
INFO:dcm2bids.dcm2bids:config: /mnt/o/AON2bids/03_bids_scaffold/code/dcm2bids_config.json
INFO:dcm2bids.dcm2bids:BIDS directory: /mnt/o/AON2bids/03_bids_scaffold/output
INFO:dcm2bids.utils:Running ['dcm2niix', '-b', 'y', '-ba', 'y', '-z', 'y', '-f', '%3s_%f_%p_%t', '-o', PosixPath('/mnt/o/AON2bids/03_bids_scaffold/output/tmp_dcm2bids/sub-03'), PosixPath('/mnt/o/AON2bids/01_data/mri/mri/EV03')]
INFO:dcm2bids.dcm2niix:Check log file for dcm2niix output
INFO:dcm2bids.sidecar

### Let's look at the output folder

In [7]:
%%bash
outputdir='/mnt/o/AON2bids/03_bids_scaffold/output/'
# delete temporary files
rm -r ${outputdir}tmp_dcm2bids
# display our BIDS-structured folder
tree $outputdir

/mnt/o/AON2bids/03_bids_scaffold/output/
├── sub-01
│   ├── anat
│   │   ├── sub-01_T1w.json
│   │   └── sub-01_T1w.nii.gz
│   ├── fmap
│   │   ├── sub-01_magnitude1.json
│   │   ├── sub-01_magnitude1.nii.gz
│   │   ├── sub-01_magnitude2.json
│   │   ├── sub-01_magnitude2.nii.gz
│   │   ├── sub-01_phasediff.json
│   │   └── sub-01_phasediff.nii.gz
│   └── func
│       ├── sub-01_task-AON_run-01_bold.json
│       ├── sub-01_task-AON_run-01_bold.nii.gz
│       ├── sub-01_task-AON_run-02_bold.json
│       ├── sub-01_task-AON_run-02_bold.nii.gz
│       ├── sub-01_task-action_bold.json
│       └── sub-01_task-action_bold.nii.gz
├── sub-02
│   ├── anat
│   │   ├── sub-02_T1w.json
│   │   └── sub-02_T1w.nii.gz
│   ├── fmap
│   │   ├── sub-02_magnitude1.json
│   │   ├── sub-02_magnitude1.nii.gz
│   │   ├── sub-02_magnitude2.json
│   │   ├── sub-02_magnitude2.nii.gz
│   │   ├── sub-02_phasediff.json
│   │   └── sub-02_phasediff.nii.gz
│   └── func
│       ├── sub-02_task-AON_run-01_bold.json
│ 